In [1]:
from model_CNN_source import *
dump_at = '/proj/dump/CNN_two_streams'
source_data = '/proj/source_data/Training_Data'
source_data_2 = '/proj/source_data/Testing_Data'
processed_data = '/proj/processed_data'

# run paths
run_paths = set_dirs(dump_at)

base folder at /proj/dump/CNN_two_streams/run_2023_01_12, 
callbacks at /proj/dump/CNN_two_streams/run_2023_01_12/callback_data, 
predictions at /proj/dump/CNN_two_streams/run_2023_01_12/pred, 
model at /proj/dump/CNN_two_streams/run_2023_01_12/model, 
tmp at /proj/dump/CNN_two_streams/run_2023_01_12/tmp_data


In [2]:
# Load data
geno_data = pd.read_csv(f"{processed_data}/geno_processed.miss.1.mac.1.biallelic.txt")
other_data = pd.read_csv(f"{processed_data}/combined_mat_v2.csv")
cv_data = read_json(f"{processed_data}/train_test_split_v3.json")

/tmp/ipykernel_59/2411887660.py:3: DtypeWarning: Columns (6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  other_data = pd.read_csv(f"{processed_data}/combined_mat_v2.csv")


In [3]:
# Identify data sources
all_cols = other_data.columns.values
data_identifier_cols = [x for x in all_cols if "Env" in x][1:]
data_identifier_cols_sl = [x for x in all_cols if "sl_" in x]
data_identifier_cols_ec = [x for x in all_cols if "ec_" in x]
data_identifier_cols_wt = [x for x in all_cols if "wt_" in x]
data_identifier_bcols_non_pheno = data_identifier_cols_sl + data_identifier_cols_ec + data_identifier_cols_wt
data_identifier_cols_non_pheno = [x for x in data_identifier_bcols_non_pheno if x not in data_identifier_cols] # these can be used as explanatory variables

In [4]:
# Create response and predictor data

## define useful columns
useful_cols = ['Env', 'Loc', 'Year', 'Hybrid', 'Yield_Mg_ha'] + data_identifier_cols_non_pheno # in the test data we only get Env and Hybrid and need to predict yield_mg_ha
pheno_data = other_data.loc[:, useful_cols] 
non_float_or_int = pd.DataFrame(pheno_data.dtypes[pheno_data.dtypes == "object"]).reset_index().loc[:, "index"].values.tolist()[3:]

## remove columns which are string only
useful_cols = [x for x in useful_cols if x not in non_float_or_int]

## define a sample set for tuning hyperparameters
test_index = np.sort(random.sample(pheno_data.index.tolist(), int(0.1*pheno_data.shape[0])))
non_test = pheno_data.loc[~pheno_data.index.isin(test_index), :].index.tolist()
val_index = np.sort(random.sample(non_test, int(0.1*len(non_test))))
train_index = np.sort([x for x in non_test if x not in val_index])


train = pheno_data.loc[train_index, useful_cols]
train['type'] = 'train'
val = pheno_data.loc[val_index, useful_cols]
val['type'] = 'val'
test = pheno_data.loc[test_index, useful_cols]
test['type'] = 'test'
phenoGE = pd.concat([train, val, test])

## scale pheno data
phenoGE_scaled, phenoGE_scaler = scale_data(phenoGE.loc[:, "Yield_Mg_ha"].values.reshape(-1, 1), ["Yield_Mg_ha"], phenoGE.index)
phenoGE['y'] = phenoGE_scaled

In [ ]:
# cnn with two streams i.e. genetic and ec

g_data_npy_path = f"{run_paths['tmp_at']}/g_data.npy"
ec_data_npy_path = f"{run_paths['tmp_at']}/ec_data.npy"

## Generate a array for genomic data
if not Path(g_data_npy_path).is_file():
    
    geno_data_scaled, geno_data_scaler = scale_data(geno_data.iloc[:, 1:], geno_data.columns[1:], geno_data.index)
    
    geno_data_npy = []
    
    for hyb in phenoGE.Hybrid:
        geno_data_npy.append(geno_data_scaled.loc[geno_data.Hybrid == hyb, :].values[0][1:].tolist())
    
    geno_data_npy_stacked = np.stack(geno_data_npy)
    geno_data_npy_stacked_reshaped = geno_data_npy_stacked.reshape(geno_data_npy_stacked.shape[0], geno_data_npy_stacked.shape[1], 1) # of dimension = (samples, markers, 1) for one dimentional convulution
    np.save(g_data_npy_path, geno_data_npy_stacked_reshaped)
else:
    geno_data_npy_stacked_reshaped = np.load(g_data_npy_path)

## Generate a stream for ec data
if not Path(ec_data_npy_path).is_file():

    phenoGE_ec_scaled, phenoGE_ec_scaler = scale_data(phenoGE.loc[:, data_identifier_cols_ec[1:]], data_identifier_cols_ec[1:], phenoGE.index)
    
    ec_data_npy = []
    
    for env in phenoGE.Env:
        ec_data_npy.append(phenoGE_ec_scaled.loc[phenoGE.Env == env, data_identifier_cols_ec[1:]].iloc[0, :].values.tolist())
    
    ec_data_npy_stacked = np.stack(ec_data_npy)
    ec_data_npy_stacked_reshaped = ec_data_npy_stacked.reshape(ec_data_npy_stacked.shape[0], ec_data_npy_stacked.shape[1], 1) # of dimension = (samples, env_days, 1) for one dimentional convulution
    
    ## save for later pickup
    np.save(ec_data_npy_path, ec_data_npy_stacked_reshaped)
else:
    ec_data_npy_stacked_reshaped = np.load(ec_data_npy_path)

X = [geno_data_npy_stacked_reshaped, ec_data_npy_stacked_reshaped]

In [ ]:
# define data for model 
X_train, X_val, X_test = [geno_data_npy_stacked_reshaped[train_index], ec_data_npy_stacked_reshaped[train_index]], \
                         [geno_data_npy_stacked_reshaped[val_index], ec_data_npy_stacked_reshaped[val_index]], \
                         [geno_data_npy_stacked_reshaped[test_index], ec_data_npy_stacked_reshaped[test_index]]

y_train, y_val, y_test = phenoGE.iloc[train_index, -1], \
                         phenoGE.iloc[val_index, -1], \
                         phenoGE.iloc[test_index, -1]

In [8]:
# perform tuning
tuning_save_at = f"{run_paths['model_at']}"

start_time_tuning = time.time()
stop_early = EarlyStopping(monitor='val_loss', patience=5, min_delta = 0.0001)

tuner = kt.Hyperband(hypermodel=model_tuner_two_streams,
                     objective=kt.Objective("val_mean_squared_error", direction="min"),
                     max_epochs=100,
                     factor=4,
                     hyperband_iterations=1,
                     overwrite = True,
                     directory=tuning_save_at,
                     project_name="hp_tuning",
                     seed=30)
tuner.search(X_train, y_train, 
             epochs=100,
             validation_data=(X_val, y_val),
             callbacks=[stop_early],
             batch_size = 64) #for non interactive - verbose - 0
#top3_params = tuner.get_best_hyperparameters(num_trials=3)

## generate model with best parameters
best_model = tuner.get_best_models()[0]
best_model.save(f"{run_paths['model_at']}/best_model")

Trial 161 Complete [01h 24m 09s]
val_mean_squared_error: 0.008596484549343586

Best val_mean_squared_error So Far: 0.008286920376121998
Total elapsed time: 1d 20h 31m 48s
INFO:tensorflow:Oracle triggered exit


2023-01-06 11:41:50.087130: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /proj/dump/CNN_two_streams/run_2023_01_04/model/best_model/assets


INFO:tensorflow:Assets written to: /proj/dump/CNN_two_streams/run_2023_01_04/model/best_model/assets


In [ ]:
# load model
best_model_loaded = tf.keras.models.load_model(f"{run_paths['model_at']}/best_model")

In [12]:
# fit model
tb_filepath = run_paths['tb_cb']
cp_filepath = run_paths['mc_cb']

## set call backs
tensorboard_cb = TensorBoard(tb_filepath)
modelcheck_cb = ModelCheckpoint(filepath=cp_filepath,
                                save_weights_only=True,
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True)
model_cb = EarlyStopping(monitor='val_loss',
                                 min_delta=0.00001,
                                 patience=5,
                                 verbose=0,
                                 mode='min',
                                 baseline=None,
                                 restore_best_weights=True)
best_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                batch_size = 64,
                epochs = 100,
                verbose = 1,
                shuffle = True,
                callbacks=[modelcheck_cb, 
                           tensorboard_cb,
                           model_cb])

best_model.load_weights(cp_filepath)

Epoch 1/100
598/598 [==============================] - 218s 364ms/step - loss: 0.0724 - mean_squared_error: 0.0088 - val_loss: 0.0667 - val_mean_squared_error: 0.0076
Epoch 2/100
598/598 [==============================] - 216s 362ms/step - loss: 0.0705 - mean_squared_error: 0.0084 - val_loss: 0.0657 - val_mean_squared_error: 0.0075
Epoch 3/100
598/598 [==============================] - 215s 359ms/step - loss: 0.0696 - mean_squared_error: 0.0082 - val_loss: 0.0662 - val_mean_squared_error: 0.0076
Epoch 4/100
598/598 [==============================] - 216s 361ms/step - loss: 0.0687 - mean_squared_error: 0.0080 - val_loss: 0.0656 - val_mean_squared_error: 0.0074
Epoch 5/100
598/598 [==============================] - 216s 361ms/step - loss: 0.0684 - mean_squared_error: 0.0080 - val_loss: 0.0653 - val_mean_squared_error: 0.0074
Epoch 6/100
598/598 [==============================] - 215s 359ms/step - loss: 0.0676 - mean_squared_error: 0.0079 - val_loss: 0.0657 - val_mean_squared_error: 0.007

In [ ]:
# perform predictions
prediction = best_model.predict(X_test)

## re-scale data
obs = inverse_scale(phenoGE_scaler, y_test, verbose = False)
pred = inverse_scale(phenoGE_scaler, prediction, verbose = False)
out_data = pd.DataFrame([test_index, obs, pred], index=["index","Observed","Predicted"]).T
out_data["index"] = out_data["index"].astype('int')

## check rmse